# Module 05: Templates

**Estimated Time:** 2 hours  
**Difficulty:** Beginner

---

## Learning Objectives

By the end of this module, you will:

- ✅ Understand Django Template Language (DTL)
- ✅ Create and organize templates
- ✅ Implement template inheritance
- ✅ Use template variables, tags, and filters
- ✅ Pass context data from views to templates
- ✅ Build reusable template components

---

## 1. Introduction to Django Templates

Templates are the presentation layer in Django's MVT pattern. They:
- Generate dynamic HTML
- Display data from views
- Provide template inheritance for DRY code
- Use Django Template Language (DTL) syntax

### Template Syntax

**Variables**: `{{ variable }}`  
**Tags**: `{% tag %}`  
**Filters**: `{{ variable|filter }}`  
**Comments**: `{# comment #}`

In [ ]:
import os
from pathlib import Path

# Setup paths
notebook_dir = Path.cwd()
project_path = notebook_dir.parent / "projects" / "myblog"
blog_app = project_path / "blog"
templates_dir = blog_app / "templates" / "blog"

# Create templates directory
templates_dir.mkdir(parents=True, exist_ok=True)

print(f"Project: {project_path}")
print(f"Templates directory: {templates_dir}")
print(f"Directory created: {templates_dir.exists()}")

## 2. Base Template (Template Inheritance)

Template inheritance allows you to create a base template and extend it in child templates.

In [ ]:
# Create base template
base_template = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}My Blog{% endblock %}</title>
    
    {# CSS Block #}
    {% block css %}
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            line-height: 1.6;
        }
        header {
            background: #333;
            color: white;
            padding: 1rem;
            margin-bottom: 2rem;
        }
        nav a {
            color: white;
            text-decoration: none;
            margin-right: 1rem;
        }
        .post {
            border-bottom: 1px solid #ddd;
            padding: 1rem 0;
        }
        footer {
            margin-top: 2rem;
            padding-top: 1rem;
            border-top: 1px solid #ddd;
            text-align: center;
        }
    </style>
    {% endblock %}
</head>
<body>
    {# Header #}
    <header>
        <h1>My Blog</h1>
        <nav>
            <a href="{% url 'blog:home' %}">Home</a>
            <a href="{% url 'blog:about' %}">About</a>
        </nav>
    </header>
    
    {# Messages #}
    {% if messages %}
        {% for message in messages %}
            <div class="message {{ message.tags }}">
                {{ message }}
            </div>
        {% endfor %}
    {% endif %}
    
    {# Main Content #}
    <main>
        {% block content %}
        {# Child templates will override this block #}
        {% endblock %}
    </main>
    
    {# Footer #}
    <footer>
        <p>&copy; 2024 My Blog. All rights reserved.</p>
        {% block footer %}{% endblock %}
    </footer>
    
    {# JavaScript Block #}
    {% block javascript %}{% endblock %}
</body>
</html>
"""

with open(templates_dir / "base.html", "w") as f:
    f.write(base_template)

print("✓ Base template created: base.html")
print("\nTemplate blocks defined:")
print("  - title: Page title")
print("  - css: Styles")
print("  - content: Main content (required)")
print("  - footer: Footer content")
print("  - javascript: Scripts")

## 3. Home Page Template

Let's create a template that extends the base template.

In [ ]:
# Create home template
home_template = """{% extends 'blog/base.html' %}

{% block title %}{{ title }} - My Blog{% endblock %}

{% block content %}
<h2>Recent Posts</h2>

{% if posts %}
    {% for post in posts %}
    <article class="post">
        <h3>
            <a href="{% url 'blog:post_detail' post.slug %}">
                {{ post.title }}
            </a>
        </h3>
        
        <p class="meta">
            By {{ post.author.username }} | 
            {{ post.publish_date|date:"F d, Y" }} |
            {{ post.views }} view{{ post.views|pluralize }}
        </p>
        
        <p>{{ post.content|truncatewords:50 }}</p>
        
        {% if post.categories.all %}
        <p class="categories">
            Categories:
            {% for category in post.categories.all %}
                <a href="{% url 'blog:category_posts' category.slug %}">
                    {{ category.name }}</a>{% if not forloop.last %}, {% endif %}
            {% endfor %}
        </p>
        {% endif %}
        
        <a href="{% url 'blog:post_detail' post.slug %}">Read more &rarr;</a>
    </article>
    {% endfor %}
    
    {# Pagination #}
    {% if is_paginated %}
    <div class="pagination">
        {% if page_obj.has_previous %}
            <a href="?page={{ page_obj.previous_page_number }}">&laquo; Previous</a>
        {% endif %}
        
        <span>Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}</span>
        
        {% if page_obj.has_next %}
            <a href="?page={{ page_obj.next_page_number }}">Next &raquo;</a>
        {% endif %}
    </div>
    {% endif %}
{% else %}
    <p>No posts available yet.</p>
{% endif %}
{% endblock %}
"""

with open(templates_dir / "home.html", "w") as f:
    f.write(home_template)

print("✓ Home template created: home.html")

## 4. Post Detail Template

In [ ]:
# Create post detail template
post_detail_template = """{% extends 'blog/base.html' %}

{% block title %}{{ post.title }} - My Blog{% endblock %}

{% block content %}
<article class="post-detail">
    <h2>{{ post.title }}</h2>
    
    <p class="meta">
        By <strong>{{ post.author.username }}</strong> |
        Published {{ post.publish_date|date:"F d, Y" }} |
        {{ post.views }} view{{ post.views|pluralize }}
    </p>
    
    {% if post.categories.all %}
    <p class="categories">
        <strong>Categories:</strong>
        {% for category in post.categories.all %}
            <a href="{% url 'blog:category_posts' category.slug %}">
                {{ category.name }}</a>{% if not forloop.last %}, {% endif %}
        {% endfor %}
    </p>
    {% endif %}
    
    <hr>
    
    <div class="content">
        {{ post.content|linebreaks }}
    </div>
    
    <hr>
    
    <p>
        <a href="{% url 'blog:home' %}">&larr; Back to posts</a>
    </p>
</article>
{% endblock %}
"""

with open(templates_dir / "post_detail.html", "w") as f:
    f.write(post_detail_template)

print("✓ Post detail template created: post_detail.html")

## 5. Category Posts Template

In [ ]:
# Create category template
category_template = """{% extends 'blog/base.html' %}

{% block title %}{{ category.name }} - My Blog{% endblock %}

{% block content %}
<h2>Category: {{ category.name }}</h2>

{% if category.description %}
<p class="description">{{ category.description }}</p>
{% endif %}

<hr>

{% if posts %}
    <p>{{ posts|length }} post{{ posts|length|pluralize }} in this category</p>
    
    {% for post in posts %}
    <article class="post">
        <h3>
            <a href="{% url 'blog:post_detail' post.slug %}">
                {{ post.title }}
            </a>
        </h3>
        <p class="meta">
            {{ post.publish_date|date:"M d, Y" }} | {{ post.views }} views
        </p>
        <p>{{ post.content|truncatewords:30 }}</p>
        <a href="{% url 'blog:post_detail' post.slug %}">Read more</a>
    </article>
    {% endfor %}
{% else %}
    <p>No posts in this category yet.</p>
{% endif %}

<p><a href="{% url 'blog:home' %}">&larr; All posts</a></p>
{% endblock %}
"""

with open(templates_dir / "category_posts.html", "w") as f:
    f.write(category_template)

print("✓ Category template created: category_posts.html")

## 6. About Page Template

In [ ]:
# Create about template
about_template = """{% extends 'blog/base.html' %}

{% block title %}About - My Blog{% endblock %}

{% block content %}
<h2>About This Blog</h2>

<p>
    Welcome to My Blog! This is a Django-powered blog built as part of 
    learning Django fundamentals.
</p>

<h3>Features</h3>
<ul>
    <li>Blog posts with categories</li>
    <li>Author attribution</li>
    <li>View counting</li>
    <li>Clean, responsive design</li>
</ul>

<h3>Technologies Used</h3>
<ul>
    <li>Django 4.2+</li>
    <li>Python 3.8+</li>
    <li>SQLite Database</li>
</ul>

<p><a href="{% url 'blog:home' %}">&larr; Back to home</a></p>
{% endblock %}
"""

with open(templates_dir / "about.html", "w") as f:
    f.write(about_template)

print("✓ About template created: about.html")

## 7. Template Tags & Filters

### Common Template Tags

| Tag | Purpose | Example |
|-----|---------|--------|
| `{% if %}` | Conditional | `{% if user.is_authenticated %}` |
| `{% for %}` | Loop | `{% for post in posts %}` |
| `{% url %}` | Generate URL | `{% url 'blog:home' %}` |
| `{% block %}` | Template inheritance | `{% block content %}` |
| `{% extends %}` | Extend base template | `{% extends 'base.html' %}` |
| `{% include %}` | Include template | `{% include 'header.html' %}` |
| `{% load %}` | Load template tags | `{% load static %}` |

### Common Filters

| Filter | Purpose | Example |
|--------|---------|--------|
| `date` | Format date | `{{ post.created|date:"Y-m-d" }}` |
| `truncatewords` | Truncate | `{{ text|truncatewords:50 }}` |
| `length` | Get length | `{{ posts|length }}` |
| `default` | Default value | `{{ value|default:"N/A" }}` |
| `lower` | Lowercase | `{{ name|lower }}` |
| `pluralize` | Pluralize | `{{ count }} item{{ count|pluralize }}` |
| `linebreaks` | Convert to paragraphs | `{{ text|linebreaks }}` |

## 8. Template Variables

### Variable Syntax
```django
{{ variable }}              # Simple variable
{{ object.attribute }}      # Object attribute
{{ dict.key }}             # Dictionary key
{{ list.0 }}               # List index
{{ object.method }}        # Call method (no args)
```

### Context from Views
```python
# In view
context = {
    'posts': Post.objects.all(),
    'title': 'Home',
    'user_count': 100,
}
return render(request, 'template.html', context)
```

```django
<!-- In template -->
{{ title }}
{{ user_count }}
{% for post in posts %}
    {{ post.title }}
{% endfor %}
```

## 9. Template Organization Best Practices

### Directory Structure
```
templates/
└── blog/
    ├── base.html              # Base template
    ├── home.html              # Home page
    ├── post_detail.html       # Post detail
    ├── category_posts.html    # Category listing
    ├── about.html             # About page
    └── includes/              # Reusable components
        ├── _header.html
        ├── _footer.html
        └── _post_card.html
```

### Naming Conventions
- Use lowercase with underscores
- Prefix partials with underscore: `_header.html`
- Group related templates in subdirectories
- Use descriptive names

## 10. Custom Template Filters (Advanced)

You can create custom filters for your templates:

```python
# blog/templatetags/blog_tags.py
from django import template

register = template.Library()

@register.filter
def reading_time(text):
    """Calculate reading time in minutes"""
    words = len(text.split())
    minutes = words / 200  # Average reading speed
    return round(minutes)
```

```django
{% load blog_tags %}
Reading time: {{ post.content|reading_time }} minutes
```

## 11. Summary & Next Steps

### What We Accomplished

✅ Created base template with inheritance  
✅ Built home page template  
✅ Created post detail template  
✅ Built category listing template  
✅ Added about page  
✅ Learned template tags and filters  
✅ Implemented proper template organization  

### Templates Created
1. `base.html` - Base template with common layout
2. `home.html` - Post listing with pagination
3. `post_detail.html` - Single post view
4. `category_posts.html` - Category-filtered posts
5. `about.html` - About page

### What's Next

In **Module 06**, we'll:
- Create Django forms
- Build ModelForms
- Implement form validation
- Handle form submissions
- Add CSRF protection
- Create post creation/editing forms

---

**Excellent! Your blog now has a complete presentation layer. Continue to Module 06!** 🎨